In [14]:
import openpyxl
import logging

wb = openpyxl.load_workbook("data/计算资产总价值test foryifei 测试1.xlsx")
ws = wb.active


In [15]:
rowNumber=ws.max_row
keywordTable={}
keywordRow=ws[1]
for index in range(len(keywordRow)):
    keywordTable[keywordRow[index].value]=index+1

In [16]:
# 首先初始化所有计算后价值为单独价值
## 当然首先要开一个新col
calculatedValueIndex=ws.max_column+1 # 最后新开一列
if "合并后资产价值" in keywordTable:
    # 如果已经存在的话就覆盖重新算一次
    calculatedValueIndex=keywordTable["合并后资产价值"]
ws.cell(row=1,column=calculatedValueIndex,value="合并后资产价值")
for i in range(2,rowNumber):
    myValue=ws.cell(row=i,column=keywordTable["单价/元"]).value
    ws.cell(row=i,column=calculatedValueIndex,value=myValue)

In [17]:
parentAssetSet=set()
parentAssetTable={} # 为了更快地合并自资产的价值，建立一个父资产row_index的字典
parentIndex=keywordTable["父资产编号"]
AssetIndex=keywordTable["资产编号"]
# 这个循环找到所有的负资产
for i in range(2,rowNumber):
    cellValue=ws.cell(row=i,column=parentIndex).value
    if cellValue not in parentAssetSet:
        parentAssetSet.add(cellValue)
# 记录所有负资产的行号
for i in range(2,rowNumber):
    cellValue:str=str(ws.cell(row=i,column=AssetIndex).value)
    # 如果是-1这种的要把后缀全部忽略掉
    # if "-" in cellValue:
    #     cellValue=cellValue.split("-")[0]
    if cellValue in parentAssetSet:
        parentAssetTable[cellValue]=i
## 然后找到所有「是否为配件/增值」为「是」的配件
isExtra=keywordTable["是否为配件/增值"]
for i in range(2,rowNumber):
    if ws.cell(row=i,column=isExtra).value=="是":
        parentAsset=ws.cell(row=i,column=parentIndex).value # 我的父资产编号
        if parentAsset not in parentAssetTable:
            logging.error(f'子资产没有找到对应的父资产{parentAsset}!!')
            continue
        mergedValue=ws.cell(row=parentAssetTable[parentAsset],column=calculatedValueIndex).value+ws.cell(row=i,column=keywordTable["单价/元"]).value
        ws.cell(row=parentAssetTable[parentAsset],column=calculatedValueIndex,value=mergedValue)
        ws.cell(row=i,column=calculatedValueIndex,value=0)

In [18]:
from datetime import datetime

def get_current_datetime():
    now = datetime.now()
    formatted_datetime = now.strftime("%m%d-%H%M%S")
    return formatted_datetime

In [19]:
wb.save(f'test_openpyxl_{get_current_datetime()}.xlsx')

In [13]:
# 测试代码块